### 机器翻译介绍

机器翻译任务就是将一个语言翻译成另一个语言，比如将一句英语翻译成法语

机器翻译是典型的 seq to seq 任务，即用一个序列来输出另一个序列

其数据集的数据部分和标签部分都是序列（比如：数据部分是英语句子，标签部分是对应的法语句子）

<br>

### 模拟数据

In [55]:
import torch
import numpy as np

data = [['go .', 'va !'], 
        ['hi .', 'salut !'],
        ['run !', 'cours !'],
        ['run !', 'courez !'],
        ['who ?', 'qui ?'],
        ['wow !', 'ça alors !']]

data = np.array(data)

<br>

### 数据集处理

分离数据部分与标签部分

In [56]:
X = data[:, 0]    # 英语
Y = data[:, 1]    # 法语

<br>

分别对数据部分和标签部分进行词元化（此处我们将每一个单词视作一个词元，每一个标点也是一个词元）

词元化相当于将一句话分割成由词元组成的序列，那么每一句话都将使用一个列表来表示

In [57]:
X_ = []
for x in X:
    X_.append(x.split())

Y_ = []
for y in Y:
    Y_.append(y.split())

X = X_
Y = Y_

<br>

分别为数据部分和标签部分建立词表（词元与数值的映射）

词表中需要添加一下四个词元：

- '\<bos>' ==> 表示序列的开始

- '\<eos>' ==> 表示序列的结束

- '\<pad>' ==> 表示序列的填充

- '\<unk>' ==> 表示未知词元

`构建词表的操作参考 14.语言模型数据预处理 ，此处不进行代码实现`

In [58]:
# 数据部分的词表（英语词表）
token_to_idx1 = {}
idx_to_token1 = {}

# 标签部分的词表（法语词表）
token_to_idx2 = {}
idx_to_token2 = {}

<br>

依据词表，将数据部分和标签部分分别映射为数值数组（非独热编码形式）

同时依据指定的 seq_len 对序列进行填充和截断，填充使用 '\<pad>' 词元

并记录每一个序列的有效长度（填充长度不算有效长度）

对于不在词表中的值，统一使用 0 来表示 ('\<unk>')

'\<bos>' 和 '\<eos>' 在数据预处理时不需要添加，针对不同的训练会有不同的添加方式

In [59]:
def build_tensor(data, token_to_idx, seq_len):
    output = []
    for line in data:
        # 长度不够进行填充
        if len(line) < seq_len:
            line = list(line) + ['<pad>'] * (seq_len - len(line))

        # 长度过长，直接截断（不考虑）
        l = []
        for i in range(seq_len):
            l.append(token_to_idx.get(line[i], 0))
        output.append(l)
    return torch.tensor(output)

In [60]:
# 数据部分和标签部分的序列长度可以不一样
seq_len1 = 5
seq_len2 = 6

X = build_tensor(X, token_to_idx1, seq_len1)
Y = build_tensor(Y, token_to_idx2, seq_len2)